# A paper study(Motivation example 2-MILP)
Li, Z., Ding, R., & Floudas, C. A. (2011). A comparative theoretical and computational study on robust counterpart optimization: I. Robust linear optimization and robust mixed integer linear optimization. Industrial &amp; Engineering Chemistry Research, 50(18), 10567–10603. https://doi.org/10.1021/ie200150p

## A generic formulation:

\begin{align*}
    \textbf{max} & \sum_{m} c_{m}x_{m} + \sum_{k} d_k y_k \\
    s.t. & \sum_{m} \tilde a_{im}x_{m} + \sum_{k} \tilde b_{ik} y_k \le \tilde p_i \quad \forall i \in I\\
    & x_j \ge 0 \\
\end{align*}

## Motivating example 2

\begin{align*}
    \textbf{max} & 3x_1 + 2x_2 - 10y_1 - 5y_2 \\
    s.t. & x_1 + x_2 \le 20 \\
    & x_1 + 2x_2 \le 12 \\
    & a_{31}x_1 + b_{31}y_1 \le 0 \\
    & a_{42}x_2 + b_{42}y_2 \le 0 \\
    & x_1 - x_2 \le -4 \\
    & 0 \le x_1, x_2 \le 10, y_1,y_2 \in \{0,1\}\\
\end{align*}

Where

\begin{align*}
    &\tilde{a}_{31} = 1 + 0.1\xi_{31} , \quad &&\tilde{b}_{31} = -20 + 2\xi_{33} \\
    &\tilde{a}_{42} = 1 + 0.1\xi_{42} , \quad &&\tilde{b}_{42} = -20 + 2\xi_{44} \\
\end{align*}

for independent random varibales $\xi_{11}$, $\xi_{12}$, $\xi_{21}$, $\xi_{22}$ in [-1, 1\] (10\% perturbation).

## A determinisitc implementation

In [38]:
import csv

from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fsolve

In [39]:
data = {}
with open('ro2_pars.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        for key, val in row.items():
            if key not in data:
                data[key] = []
            data[key].append(val)
data

{'': ['obj', '1', '2', '3', '4', '5'],
 'x1': ['3', '1', '1', '1', '0', '1'],
 'x2': ['2', '1', '2', '0', '1', '-1'],
 'y1': ['-10', '0', '0', '-20', '0', '0'],
 'y2': ['-5', '0', '0', '0', '-20', '0'],
 'p': ['', '20', '12', '0', '0', '-4']}

In [40]:
# Extract obj values
c_values = {i: float(data["x"+str(i)][0]) for i in range(1, 3)}
d_values = {i: float(data["y"+str(i)][0]) for i in range(1, 3)}

# Extract coefficient values
a_values = {(i, j): float(data["x"+str(j)][i]) for i in range(1, 6) for j in range(1, 3)}
b_values = {(i, j): float(data["y"+str(j)][i]) for i in range(1, 6) for j in range(1, 3)}
p_values = {i: float(data["p"][i]) for i in range(1, 6)}

In [41]:
# Implementation of the deterministic form without uncertainty

m1 = ConcreteModel()
m1.i = Set(initialize=range(1,6), doc='Constraint index')
m1.m = Set(initialize=[1,2], doc='Variable index(continuous)')
m1.k = Set(initialize=[1,2], doc='Variable index(discrete)')

m1.a = Param(m1.i, m1.m, initialize=a_values, doc='Variable coef(continuous)')
m1.b = Param(m1.i, m1.k, initialize=b_values, doc='Variable coef(discrete)')
m1.c = Param(m1.m, initialize=c_values, doc='Objective coef(continuous)')
m1.d = Param(m1.k, initialize=d_values, doc='Objective coef(discrete)')
m1.p = Param(m1.i, initialize=p_values, doc='Constraint constant')

m1.x = Var(m1.m,  domain=NonNegativeReals)
m1.y = Var(m1.k,  domain=Binary)

def constraints_rule(model, i):
    return sum(model.a[i, m] * model.x[m] for m in model.m) + sum(model.b[i, k] * model.y[k] for k in model.k) <= model.p[i]

m1.constraints = Constraint(m1.i, rule=constraints_rule)

m1.obj = Objective(expr = sum(m1.c[m] * m1.x[m] + m1.d[k] * m1.y[k] for (m, k) in zip(m1.m, m1.k)), sense=maximize)

opt1 = SolverFactory("gurobi_persistent")
opt1.set_instance(m1)
res1 = opt1.solve()
m1.x.pprint()
m1.y.pprint()
print(value(m1.obj))

x : Size=2, Index=m
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :  None : False : False : NonNegativeReals
      2 :     0 :   6.0 :  None : False : False : NonNegativeReals
y : Size=2, Index=k
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :     1 : False : False : Binary
      2 :     0 :   1.0 :     1 : False : False : Binary
7.0
